In [1]:
import pandas as pd
import numpy as np
import os
from sqlalchemy import create_engine

In [6]:
try:
    os.path.exists('datasets/telecomm_csi.db')
    path_to_db = 'datasets/telecomm_csi.db'
    engine = create_engine(f'sqlite:///{path_to_db}', echo = False)
except:
    path_to_db = '/datasets/telecomm_csi.db'
    engine = create_engine(f'sqlite:///{path_to_db}', echo = False)

In [7]:
query = """
WITH
    u AS (
        SELECT *,
            CASE
                WHEN lt_day >= 365 THEN 'Нет'
                WHEN lt_day < 365 THEN 'Да'
            END AS is_new,
            CASE
                WHEN nps_score <= 6 THEN 'Критик'
                WHEN nps_score >= 9 THEN 'Сторонник'
                WHEN nps_score > 6 AND nps_score < 9 THEN 'Нейтрал'
            END AS nps_group,
            CASE
                WHEN gender_segment = 1 THEN 'Ж'
                WHEN gender_segment = 0 THEN 'М'
            END AS gender_seg
        FROM user),
    l AS (SELECT * FROM location),
    a AS (SELECT * FROM age_segment),
    ts AS (SELECT * FROM traffic_segment),
    ls AS (SELECT * FROM lifetime_segment)



SELECT u.user_id,
       u.lt_day,
       u.is_new,
       u.age,
       u.gender_seg AS gender_segment,
       u.os_name,
       u.cpe_type_name,
       l.country,
       l.city,
       substring(a.title, instr(a.title, ' ')) AS age_segment,
       substring(ts.title, instr(ts.title, ' '))  AS traffic_segment,
       substring(ls.title, instr(ls.title, ' ')) AS lifetime_segment,
       u.nps_score,
       u.nps_group

FROM u
JOIN l ON u.location_id = l.location_id
JOIN a ON u.age_gr_id = a.age_gr_id
JOIN ts ON u.tr_gr_id = ts.tr_gr_id
JOIN ls ON u.lt_gr_id = ls.lt_gr_id



"""

In [8]:
df = pd.read_sql(query, engine)
df.head(5)

,user_id,lt_day,is_new,age,gender_segment,os_name,cpe_type_name,country,city,age_segment,traffic_segment,lifetime_segment,nps_score,nps_group
0,A001A2,2320,Нет,45.0,Ж,ANDROID,SMARTPHONE,Россия,Уфа,45-54,1-5,36+,10,Сторонник
1,A001WF,2344,Нет,53.0,М,ANDROID,SMARTPHONE,Россия,Киров,45-54,1-5,36+,10,Сторонник
2,A003Q7,467,Нет,57.0,М,ANDROID,SMARTPHONE,Россия,Москва,55-64,20-25,13-24,10,Сторонник
3,A004TB,4190,Нет,44.0,Ж,IOS,SMARTPHONE,Россия,РостовнаДону,35-44,0.1-1,36+,10,Сторонник
4,A004XT,1163,Нет,24.0,М,ANDROID,SMARTPHONE,Россия,Рязань,16-24,5-10,36+,10,Сторонник


In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 502493 entries, 0 to 502492
Data columns (total 14 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   user_id           502493 non-null  object 
 1   lt_day            502493 non-null  int64  
 2   is_new            502493 non-null  object 
 3   age               501939 non-null  float64
 4   gender_segment    501192 non-null  object 
 5   os_name           502493 non-null  object 
 6   cpe_type_name     502493 non-null  object 
 7   country           502493 non-null  object 
 8   city              502493 non-null  object 
 9   age_segment       502493 non-null  object 
 10  traffic_segment   502493 non-null  object 
 11  lifetime_segment  502493 non-null  object 
 12  nps_score         502493 non-null  int64  
 13  nps_group         502493 non-null  object 
dtypes: float64(1), int64(2), object(11)
memory usage: 53.7+ MB


In [10]:
df.isna().sum()

user_id                0
lt_day                 0
is_new                 0
age                  554
gender_segment      1301
os_name                0
cpe_type_name          0
country                0
city                   0
age_segment            0
traffic_segment        0
lifetime_segment       0
nps_score              0
nps_group              0
dtype: int64

In [11]:
df.to_csv('dash_telecomm.csv', index=False)

### Укажите ссылку на дашборд на сайте Tableau Public:

https://public.tableau.com/app/profile/artem7301/viz/Dashtele/Dashboardtele